# BERT

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

In [3]:
file_path = "../data_sets/dataset_raw.csv"
data = pd.read_csv(file_path)

In [6]:
X = data['text']
y = data['category']
print(X.shape)
print(y.shape)

(20000,)
(20000,)


In [7]:
unique_values, counts = np.unique(y, return_counts=True)
value_counts = dict(zip(unique_values, counts))
print("Value Counts:")
for value, count in value_counts.items():
    print(f"{value}: {count}")

Value Counts:
0: 5797
1: 6761
2: 1641
3: 2709
4: 2373
5: 719


In [9]:
# !pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 7.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.9/170.9 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 387.4/387.4 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 346.4/346.4 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.2/24.2 MB 2.2 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 2.4 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 2.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.2/52.2 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.2/79.2 kB 2.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2022.11.0
    Uninstalling fsspec-2022.11.0:
      Successfully uninstalled fsspec-2022.11.0
  Attempting uninstall: dill
    Found existing installation: dill 0.3.6
    Uninstall

In [15]:
import torch
from datasets import load_dataset
from transformers import pipeline
import pandas as pd
import plotly.express as px

In [16]:
device = torch.device("cuda")

In [18]:
# BERT - uncased emotion
# A find-tuned bert-based classifier 
# https://huggingface.co/bhadresh-savani/bert-base-uncased-emotion

classifier = pipeline("text-classification",model='bhadresh-savani/bert-base-uncased-emotion', return_all_scores=True)
prediction = classifier("I love using transformers. The best part is wide range of support and its easy to use", )
print(prediction)

[[{'label': 'sadness', 'score': 0.0005138218402862549}, {'label': 'joy', 'score': 0.9972521662712097}, {'label': 'love', 'score': 0.0007443306967616081}, {'label': 'anger', 'score': 0.0007404911448247731}, {'label': 'fear', 'score': 0.0003293847548775375}, {'label': 'surprise', 'score': 0.0004197480739094317}]]


/Users/marklee/anaconda3/lib/python3.10/site-packages/transformers/pipelines/text_classification.py:89: UserWarning: `return_all_scores` is now deprecated,  if want a similar funcionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [ ]:
# run the model to test our dataset
# note, use the exactly same way to split the dataset!!!!!!